In [85]:
import os
import PIL
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from google.colab import files

## Load Data

In [2]:
files.upload()

Saving flower_photos.zip to flower_photos.zip


In [ ]:
!unzip flower_photos.zip

In [86]:
batch_size = 32
img_width = 227
img_height = 227

In [87]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  directory='flower_photos',
  validation_split=0.2,
  subset="training",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 2936 files for training.


In [88]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  directory='flower_photos',
  validation_split=0.2,
  subset="validation",
  seed=42,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3670 files belonging to 5 classes.
Using 734 files for validation.


In [89]:
class_names = train_ds.class_names
print(class_names)

['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']


In [90]:
for img, label in train_ds:
    print(img.shape,label.shape)
    break

(32, 227, 227, 3) (32,)


In [91]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Modeling

In [92]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer ,Conv2D, MaxPool2D, ZeroPadding2D, Flatten, Dense

model = Sequential([
                    InputLayer((227,227,3)),
                    Conv2D(96, 11, 4, activation='relu'),
                    MaxPool2D((3,3), strides=2),
                    ZeroPadding2D((2,2)),
                    Conv2D(256, 5, activation='relu'),
                    MaxPool2D((3,3), 2),
                    Conv2D(384, 3, padding='same', activation='relu'),
                    Conv2D(384, 3, padding='same', activation='relu'),
                    Conv2D(256, 3, padding='same', activation='relu'),
                    MaxPool2D((3,3), 2),
                    Flatten(),
                    Dense(4096, activation='relu'),
                    Dense(4096, activation='relu'),
                    Dense(100, activation='softmax'),
])

In [93]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [94]:
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_60 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 27, 27, 96)        0         
_________________________________________________________________
zero_padding2d_10 (ZeroPaddi (None, 31, 31, 96)        0         
_________________________________________________________________
conv2d_61 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 13, 13, 256)       0         
_________________________________________________________________
conv2d_62 (Conv2D)           (None, 13, 13, 384)       885120    
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 13, 13, 384)     

In [97]:
history = model.fit(train_ds, epochs=50, validation_data=val_ds)

Epoch 1/50
92/92 [==============================] - 4s 47ms/step - loss: 1.2364 - accuracy: 0.5099 - val_loss: 1.2253 - val_accuracy: 0.5123
Epoch 2/50
92/92 [==============================] - 4s 46ms/step - loss: 1.1562 - accuracy: 0.5317 - val_loss: 1.2489 - val_accuracy: 0.5068
Epoch 3/50
92/92 [==============================] - 4s 46ms/step - loss: 1.0959 - accuracy: 0.5569 - val_loss: 1.2040 - val_accuracy: 0.5163
Epoch 4/50
92/92 [==============================] - 4s 46ms/step - loss: 1.0964 - accuracy: 0.5606 - val_loss: 1.1638 - val_accuracy: 0.5531
Epoch 5/50
92/92 [==============================] - 4s 46ms/step - loss: 1.0728 - accuracy: 0.5603 - val_loss: 1.1432 - val_accuracy: 0.5245
Epoch 6/50
92/92 [==============================] - 4s 46ms/step - loss: 1.0007 - accuracy: 0.5984 - val_loss: 1.3437 - val_accuracy: 0.5054
Epoch 7/50
92/92 [==============================] - 4s 46ms/step - loss: 0.9457 - accuracy: 0.6144 - val_loss: 1.2699 - val_accuracy: 0.5177
Epoch 8/50
92

In [98]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.preprocessing.image.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

This image most likely belongs to tulips with a 2.67 percent confidence.
